<a href="https://colab.research.google.com/github/Riddhish1/Finance_APP/blob/main/Models/BERT_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
df = pd.read_csv('/content/bruh_updated.csv', encoding='latin-1') # Or 'iso-8859-1' or another suitable encoding

In [5]:
df.tail()

,Reason,Category,Transaction_ID,Date,Time
1122,phone,Other,DAMN123456,04-08-2025,14:45:55
1123,jeans,Other,JEANS1234X,05-08-2025,12:20:25
1124,pants,Other,P1A2TS484H,18-08-2025,17:50:10
1125,belt,Other,YAHI123855,19-08-2025,16:25:55
1126,"Shoes(Nike,Puma,Adidas,reebok,asics,under armour)",Other,NAHI123855,20-08-2025,19:30:20


In [6]:
df['Category'].unique()

array(['Food & Dining', 'Daily & Household Expenses', 'Health & Wellness',
       'Other', 'Transportation & Travel', 'Entertainment & Leisure',
       'Financial & Legal Transactions', 'Education & Learning',
       'Financial & Legal'], dtype=object)

In [7]:
df["Category"] = df["Category"].replace({
    "Financial & Legal Transactions": "Financial & Legal"
})

In [8]:
df['Category'].value_counts()

,count
Category,
Other,182
Daily & Household Expenses,179
Health & Wellness,148
Education & Learning,137
Financial & Legal,131
Transportation & Travel,122
Entertainment & Leisure,118
Food & Dining,110


In [9]:
category_mapping = {
    "Daily & Household Expenses": 0,
    "Health & Wellness": 1,
    "Other": 2,
    "Education & Learning": 3,
    "Financial & Legal": 4,
    "Transportation & Travel": 5,
    "Entertainment & Leisure": 6,
    "Food & Dining": 7
}
df["label"] = df["Category"].map(category_mapping)


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127 entries, 0 to 1126
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Reason          1127 non-null   object
 1   Category        1127 non-null   object
 2   Transaction_ID  1127 non-null   object
 3   Date            1127 non-null   object
 4   Time            1127 non-null   object
 5   label           1127 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 53.0+ KB


In [10]:
X = df['Reason']  # Features (transaction descriptions)
y = df['Category']  # Labels (categories)

In [39]:
import nltk
from nltk.corpus import stopwords
# Download stopwords if not already downloaded
nltk.download('stopwords')

# Load English stopwords
stop_words = set(stopwords.words('english'))
manual_stopwords = {
    "for", "a", "at", "in", "to", "from", "s",  # Common fillers
    "paid", "purchase", "purchased", "bought", "ordered", "ticket",
    "subscription", "fees", "charges", "fee", "service", "registration", "rental"  # Generic transaction words
}

# Combine both sets of stopwords
stop_words = stop_words.union(manual_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
df['Reason'] = df['Reason'].astype(str).apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

In [41]:
from sklearn.model_selection import train_test_split

# 80% training, 20% testing
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["Reason"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)


In [42]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # Needed for BERT preprocessing


In [43]:
import tensorflow as tf
import tensorflow_hub as hub

# Load BERT preprocessing and encoder as Keras layers
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [44]:
from transformers import BertTokenizer

In [45]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize training and testing data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")


In [46]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

In [47]:
num_labels = len(category_mapping)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
!pip install datasets

In [49]:
import torch
from datasets import Dataset

# Convert lists to Hugging Face Dataset format
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels  # Ensure labels are included
})

test_dataset = Dataset.from_dict({
    "input_ids": test_encodings["input_ids"],
    "attention_mask": test_encodings["attention_mask"],
    "labels": test_labels
})

In [50]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [51]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}


In [52]:
from transformers import Trainer
import torch.nn as nn

num_classes = 8  # Update according to your dataset
model.classifier = nn.Linear(model.config.hidden_size, num_classes)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-52-b89b65bc73c5>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [53]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.121295,0.712389
2,No log,0.753916,0.756637
3,No log,0.793399,0.743363
4,No log,0.862034,0.774336
5,0.748300,0.950260,0.765487
6,0.748300,1.101600,0.747788
7,0.748300,1.209928,0.765487
8,0.748300,1.255832,0.765487
9,0.080700,1.260064,0.765487
10,0.080700,1.235544,0.765487


TrainOutput(global_step=1130, training_loss=0.370946486650315, metrics={'train_runtime': 4169.6254, 'train_samples_per_second': 2.161, 'train_steps_per_second': 0.271, 'total_flos': 101868521494080.0, 'train_loss': 0.370946486650315, 'epoch': 10.0})

In [54]:
predictions = trainer.predict(test_dataset).predictions
predicted_labels = np.argmax(predictions, axis=1)
print(classification_report(test_labels, predicted_labels))


              precision    recall  f1-score   support

           0       0.67      0.80      0.73        30
           1       1.00      0.94      0.97        35
           2       0.59      0.74      0.66        35
           3       0.86      0.91      0.89        34
           4       0.89      0.68      0.77        25
           5       0.65      0.81      0.72        16
           6       0.75      0.52      0.61        29
           7       0.78      0.64      0.70        22

    accuracy                           0.77       226
   macro avg       0.77      0.76      0.76       226
weighted avg       0.78      0.77      0.77       226



In [55]:
# ✅ Save the Model
save_path = r"C:\Users\bonde\Desktop\models\bert-classi"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('C:\\Users\\bonde\\Desktop\\models\\bert-classi/tokenizer_config.json',
 'C:\\Users\\bonde\\Desktop\\models\\bert-classi/special_tokens_map.json',
 'C:\\Users\\bonde\\Desktop\\models\\bert-classi/vocab.txt',
 'C:\\Users\\bonde\\Desktop\\models\\bert-classi/added_tokens.json')

In [61]:
import os
if os.path.exists(save_path):
    print(f"Model saved at: {save_path}")
else:
    print("Model save path does not exist.")


Model saved at: C:\Users\bonde\Desktop\models\bert-classi


In [57]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np

# Define the path where your model is saved
model_path = r"C:\Users\bonde\Desktop\models\bert-classi"

# Load the trained model & tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Set model to evaluation mode (disables training mode)
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [58]:
def predict_category(text):
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Run input through model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label (highest probability)
    logits = outputs.logits
    predicted_label = np.argmax(logits.numpy())

    # Label mapping (Ensure this matches the one used in training)
    category_mapping = {
    0: "Daily & Household Expenses",
    1: "Health & Wellness",
    2: "Other",
    3: "Education & Learning",
    4: "Financial & Legal",
    5: "Transportation & Travel",
    6: "Entertainment & Leisure",
    7: "Food & Dining"
    }

    return category_mapping[predicted_label]


In [60]:
test_examples = [
    "Bought pizza from Domino's",
    "Paid for an Uber ride",
    "Received my monthly salary",
    "Purchased a new refrigerator",
    "Bought a pair of jeans",
    "HackHub"
]

for example in test_examples:
    print(f"Transaction: {example} → Predicted Category: {predict_category(example)}")


Transaction: Bought pizza from Domino's → Predicted Category: Food & Dining
Transaction: Paid for an Uber ride → Predicted Category: Transportation & Travel
Transaction: Received my monthly salary → Predicted Category: Other
Transaction: Purchased a new refrigerator → Predicted Category: Daily & Household Expenses
Transaction: Bought a pair of jeans → Predicted Category: Other
Transaction: HackHub → Predicted Category: Education & Learning
